# 1. Install Dependencies

In [ ]:
!pip3 install torch torchvision torchaudio

In [ ]:
#YOLOv5
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!cd yolov5 & pip install -r requirements.txt

In [ ]:
#YOLPv7
!git clone https://github.com/WongKinYiu/yolov7

In [ ]:
!cd yolov7 & pip install -r requirements.txt

In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import uuid   # Unique identifier
import os
import time

# 2. Load & Configure Model via PyTorch

In [2]:
#YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5x6')

Using cache found in C:\Users\leeju/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-12-18 Python-3.8.15 torch-1.10.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients, 209.6 GFLOPs
Adding AutoShape... 


In [ ]:
#YOLOv7
model = torch.hub.load("WongKinYiu/yolov7", 'yolov7')

In [3]:
model.conf = 0.40
model.max_det = 20
model.classes = [0,67]

# 3.1 Image Detection

In [ ]:
img = 'https://www.shutterstock.com/image-photo/sao-paulo-brazil-may-07-260nw-595810271.jpg'

results = model(img)
results.print()

%matplotlib inline 
plt.imshow(np.squeeze(results.render()))
plt.show()

# 3.2 Realtime Detection w/ Webcam

### Diary:
1. 18/12 FPS Display, Automatic Counter for GauzeIn
2. 20/12 Hand Detection pauses counter

### Things to add:
1. Saving function
2. GUI

In [ ]:
count = 0 ##
countIn = 0 ##Gauze In
condition = 5
i,j = 0,0
fps = 0
startTime, endTime = 0,0
working = 0
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # Make detections
    results = model(frame)

    # Make Counter Calculations
    a,b = 0,0
    for detection in results.xywh[0]:
        if detection[5].item() in model.classes[1:]:
            a += 1
        if detection[5].item() == 0:
            b += 1
    if b == 0:
        tempcount = a
        if tempcount != count:
            i += 1
            if i == condition:
                if tempcount > count:
                    countIn += tempcount - count
                count = tempcount
                working = countIn - count
        else:
            i = 0
        image = np.squeeze(results.render())
        cv2.putText(image, 'On Screen =' + str(count), (300,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, 'Total In =' + str(countIn), (300,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, 'Working =' + str(working), (300,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
#         cv2.putText(image, "FPS =" + str(round(fps, 1)), (50,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Gauze', image)
    else:
        image = np.squeeze(results.render())
        cv2.putText(image, 'On Screen =' + 'Paused', (300,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, 'Total In =' + 'Paused', (300,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, 'Working =' + 'Paused', (300,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
#         cv2.putText(image, "FPS =" + str(round(fps, 1)), (50,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, 'Hand Detected', (200,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Gauze', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()